In [ ]:
###############################3######################## Color Tracking Code ##############################################################
import time
import numpy as np
import cv2


cap=cv2.VideoCapture(0)

def nothing(x):
    pass
cv2.namedWindow("Trackbars")
cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 255, 255, nothing)


while True:
     ret,frame=cap.read()
     frame=cv2.resize(frame,(640,480))
     hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
     l_h = cv2.getTrackbarPos("L - H", "Trackbars")
     l_s = cv2.getTrackbarPos("L - S", "Trackbars")
     l_v = cv2.getTrackbarPos("L - V", "Trackbars")
     u_h = cv2.getTrackbarPos("U - H", "Trackbars")
     u_s = cv2.getTrackbarPos("U - S", "Trackbars")
     u_v = cv2.getTrackbarPos("U - V", "Trackbars")
     lower_blue = np.array([l_h, l_s, l_v])
     upper_blue = np.array([u_h, u_s, u_v])
     mask = cv2.inRange(hsv, lower_blue, upper_blue)
     result = cv2.bitwise_and(frame, frame, mask=mask)    

    # show thresholded image
     cv2.imshow("mask", mask)
     cv2.imshow("result", result)  

     key = cv2.waitKey(1) & 0xFF
     if key == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
####################################################### Final Distance Detect Code ################################################################
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
lower_range = np.array([65, 10, 77]) #set the initial Values 
upper_range = np.array([86, 82, 225]) #set the initial Values 

Known_distance = 40.0
Known_width = 18.0

Detected_object = 0
Object_distance = 0

def Focal_Length_Finder(Known_distance, real_width, width_in_rf_image):
    focal_length = (width_in_rf_image * Known_distance) / real_width
    return focal_length

def obj_data(img):
    obj_width = 0
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_range, upper_range)
    _, mask1 = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    cnts, _ = cv2.findContours(mask1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for c in cnts:
        x = 600
        if cv2.contourArea(c) > x:
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            obj_width = w
            return obj_width
    return 0

def Distance_finder(Focal_Length, Known_width, obj_width_in_frame):
    distance = (Known_width * Focal_Length) / obj_width_in_frame
    return distance

ref_image = cv2.imread("rf.png")
ref_image_obj_width = obj_data(ref_image)
Focal_length_found = Focal_Length_Finder(Known_distance, Known_width, ref_image_obj_width)
cv2.imshow("ref_image", ref_image)

# Arm Coding
def arm():
    pass

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (640, 480))
    obj_width_in_frame = obj_data(frame)
    if obj_width_in_frame != 0:
        Object_distance = Distance_finder(Focal_length_found, Known_width, obj_width_in_frame)
        Detected_object = 1
    else:
        Detected_object = 0
        Object_distance = 0

    #print(f"Detected object: {Detected_object}")
    #print(f"Distance: {round(Object_distance, 2)} CM")
    if (Detected_object == 1) and (25 < round(Object_distance, 2) < 30):
        #stop drone movement
        #call arm function
        arm()
        print(f"Distance: {round(Object_distance, 2)} CM")
        

    cv2.putText(frame, f"Distance: {round(Object_distance, 2)} CM", (30, 35), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 0, 0), 2)
    cv2.imshow("FRAME", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()